In [19]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

In [20]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-10-07 20:11:04.830 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\mwien\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:00<00:00, 3162.08it/s]


In [21]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))

In [22]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [23]:
loss_fn = torch.nn.CrossEntropyLoss()

In [24]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [25]:
settings = TrainerSettings(
    epochs=200, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs=None
    # earlystop_kwargs = {
    #     "save": True, # save every best model, and restore the best one
    #     "verbose": True,
    #     "patience": 40, # number of epochs with no improvement after which training will be stopped
    #     "delta": 0.0, # minimum change to be considered an improvement
    # }
)
settings

epochs: 200
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: None

In [26]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [27]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)
    
n_repeats = 1

for repeat in range(n_repeats):
    print(f"Repeat {repeat+1}/{n_repeats}")
    
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", f"{repeat}_GRU_200epochs_0.5drop_128hidsize")
        mlflow.set_tag("dev", "Marcello")
        # Log hyperparameters to MLflow

        mlflow.log_param("epochs", settings.epochs)

        mlflow.log_param("learning_rate", settings.optimizer_kwargs.get("lr", None))
        
        config = ModelConfig(
            input_size=3,
            hidden_size=128,
            num_layers=1,
            output_size=20,
            dropout=0.5,
        )

        model = GRUmodel(
            config=config,
        )

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        if not settings.earlystop_kwargs["save"]:
            tag = datetime.now().strftime("%Y%m%d-%H%M-")
            modelpath = modeldir / (tag + "model.pt")
            torch.save(model, modelpath)

c:\Master Applied Data Science\Year 2\Semester 3 (Deep Learning & Model Deployment)\Portfolio-Marcello-Wienhoven\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
2025-10-07 20:11:06.374 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251007-201106


Repeat 1/1


100%|██████████| 81/81 [00:01<00:00, 62.64it/s]
2025-10-07 20:11:07.822 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.6692 test 2.3054 metric ['0.1844']
100%|██████████| 81/81 [00:01<00:00, 64.59it/s]
2025-10-07 20:11:09.239 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.1663 test 2.0103 metric ['0.2750']
100%|██████████| 81/81 [00:01<00:00, 70.27it/s]
2025-10-07 20:11:10.541 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.8242 test 1.6907 metric ['0.4109']
100%|██████████| 81/81 [00:01<00:00, 64.81it/s]
2025-10-07 20:11:11.938 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.4152 test 1.2283 metric ['0.5422']
100%|██████████| 81/81 [00:01<00:00, 65.36it/s]
2025-10-07 20:11:13.337 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.9643 test 0.7918 metric ['0.7266']
100%|██████████| 81/81 [00:01<00:00, 60.22it/s]
2025-10-07 20:11:14.829 | INFO     | mltrainer.trainer:report:209 - Epoch 5 train 0.7128 test 0.6956 metric ['0.7656'

TypeError: 'NoneType' object is not subscriptable

In [ ]:
mlflow.end_run()

## Open MLFlow
"""
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
        
mlflow server --backend-store-uri sqlite:///3-hypertuning-rnn/mlflow.db --host 127.0.0.1 --port 5000
```
"""